In [33]:
import numpy as np
from sklearn.decomposition import PCA
import time


In [34]:
# Constants
SIGMA_SQUARED = 1.0  # Variance of prior
EPSILON = 0.01       # Learning rate
N_COMPONENTS_LIST = [10, 20, 30] # List of numbers of principal components to use


In [35]:
def load_mnist_data(folder, digits):
    """
    Load MNIST data from text files and filter for specific digits.

    Parameters:
    - folder (str): Path to the folder containing MNIST data files.
    - digits (list): List of digits to filter (e.g., [0, 1]).

    Returns:
    - X_train (np.ndarray): Training data features.
    - y_train (np.ndarray): Training data labels.
    - X_test (np.ndarray): Test data features.
    - y_test (np.ndarray): Test data labels.
    """
    # Construct file paths
    trainX_path = f'{folder}/trainX.txt'
    trainY_path = f'{folder}/trainY.txt'
    testX_path = f'{folder}/testX.txt'
    testY_path = f'{folder}/testY.txt'

    # Load data from text files with the correct delimiter
    X_train = np.loadtxt(trainX_path, delimiter=',')
    y_train = np.loadtxt(trainY_path, delimiter=',').astype(int)
    X_test = np.loadtxt(testX_path, delimiter=',')
    y_test = np.loadtxt(testY_path, delimiter=',').astype(int)

    # Filter for specified digits
    train_filter = np.isin(y_train, digits)
    test_filter = np.isin(y_test, digits)
    X_train = X_train[train_filter]
    y_train = y_train[train_filter]
    X_test = X_test[test_filter]
    y_test = y_test[test_filter]

    # Adjust labels to 0 and 1
    y_train = (y_train == digits[1]).astype(int)
    y_test = (y_test == digits[1]).astype(int)

    return X_train, y_train, X_test, y_test


In [36]:
def sigmoid(z):
    """
    Compute the sigmoid function.

    Parameters:
    - z (np.ndarray): Input array.

    Returns:
    - np.ndarray: Sigmoid of the input.
    """
    return 1 / (1 + np.exp(-z))


In [37]:
def compute_gradient(X, y, beta, sigma_squared):
    """
    Compute the gradient of the loss function U(beta).

    Parameters:
    - X (np.ndarray): Feature matrix.
    - y (np.ndarray): Labels vector.
    - beta (np.ndarray): Coefficient vector.
    - sigma_squared (float): Variance of the prior distribution.

    Returns:
    - np.ndarray: Gradient vector.
    """
    predictions = sigmoid(X @ beta)
    error = predictions - y
    gradient = X.T @ error + beta / sigma_squared
    return gradient


In [38]:
def map_logistic_regression(X, y, sigma_squared, epsilon, max_iter=5000, tol=1e-6):
    """
    Perform MAP estimation for logistic regression using gradient descent.

    Parameters:
    - X (np.ndarray): Feature matrix.
    - y (np.ndarray): Labels vector.
    - sigma_squared (float): Variance of the prior distribution.
    - epsilon (float): Learning rate (step size).
    - max_iter (int): Maximum number of iterations.
    - tol (float): Tolerance for convergence.

    Returns:
    - beta (np.ndarray): Estimated coefficients.
    """
    beta = np.zeros(X.shape[1])
    for iteration in range(max_iter):
        gradient = compute_gradient(X, y, beta, sigma_squared)
        beta_new = beta - epsilon * gradient

        # Check for convergence
        if np.linalg.norm(beta_new - beta, ord=1) < tol:
            print(f'Converged after {iteration + 1} iterations.')
            break

        beta = beta_new

    return beta


In [39]:
def predict(X, beta):
    """
    Predict class labels for given data and coefficients.

    Parameters:
    - X (np.ndarray): Feature matrix.
    - beta (np.ndarray): Coefficient vector.

    Returns:
    - np.ndarray: Predicted labels (0 or 1).
    """
    probabilities = sigmoid(X @ beta)
    return (probabilities >= 0.5).astype(int)


In [40]:
def evaluate_model(y_true, y_pred):
    """
    Evaluate the model using zero-one loss (average error rate).

    Parameters:
    - y_true (np.ndarray): True labels.
    - y_pred (np.ndarray): Predicted labels.

    Returns:
    - float: Average error rate.
    """
    error_rate = np.mean(np.abs(y_true - y_pred))
    return error_rate


In [41]:
def run_pca_experiment(folder, digits, sigma_squared, epsilon, n_components_list):
    """
    Run the PCA and MAP logistic regression experiments for specified digits.

    Parameters:
    - folder (str): Path to the folder containing MNIST data files.
    - digits (list): Digits to classify (e.g., [0, 1]).
    - sigma_squared (float): Variance of the prior distribution.
    - epsilon (float): Learning rate (step size).
    - n_components_list (list): List of numbers of principal components to use.
    """
    # Load and preprocess data
    X_train_full, y_train, X_test_full, y_test = load_mnist_data(folder, digits)

    results = []

    for n_components in n_components_list:
        print(f'\nRunning experiment with {n_components} principal components.')

        # Record the start time
        start_time = time.time()

        # Apply PCA to training data
        pca = PCA(n_components=n_components)
        X_train_reduced = pca.fit_transform(X_train_full)

        # Apply the same PCA transformation to test data
        X_test_reduced = pca.transform(X_test_full)

        # Add bias term to reduced data
        X_train = np.hstack([np.ones((X_train_reduced.shape[0], 1)), X_train_reduced])
        X_test = np.hstack([np.ones((X_test_reduced.shape[0], 1)), X_test_reduced])

        # Train the model using MAP estimation
        beta = map_logistic_regression(X_train, y_train, sigma_squared, epsilon)

        # Predict on test data
        y_pred = predict(X_test, beta)

        # Evaluate the model
        error_rate = evaluate_model(y_test, y_pred)
        accuracy = 1 - error_rate

        # Record the end time
        end_time = time.time()
        time_consumption = end_time - start_time

        # Print results
        print(f'Number of Principal Components: {n_components}')
        print(f'Accuracy: {accuracy:.4f}')
        print(f'Time Consumption: {time_consumption:.4f} seconds')
        print(f'Parameters used: sigma_squared={sigma_squared:.1f}, epsilon={epsilon:.2f}')


        # Store results
        results.append({
            'n_components': n_components,
            'accuracy': accuracy,
            'time_consumption': time_consumption
        })

    return results


In [42]:
def main():
    # Path to the data folder
    data_folder = 'data-mnist'

    # Run the PCA experiments
    results = run_pca_experiment(
        folder=data_folder,
        digits=[0, 1],
        sigma_squared=SIGMA_SQUARED,
        epsilon=EPSILON,
        n_components_list=N_COMPONENTS_LIST
    )

    # Print a summary of results
    print('\nSummary of Results:')
    for res in results:
        print(f"PC = {res['n_components']}, Accuracy = {res['accuracy']:.4f}, Time = {res['time_consumption']:.4f} seconds, Parameters used: sigma_squared={sigma_squared:.1f}, epsilon={epsilon:.2f}")

if __name__ == '__main__':
    main()



Running experiment with 10 principal components.
Converged after 615 iterations.
Number of Principal Components: 10
Accuracy: 1.0000
Time Consumption: 0.0350 seconds
Parameters used: sigma_squared=1.0, epsilon=0.01

Running experiment with 20 principal components.
Converged after 698 iterations.
Number of Principal Components: 20
Accuracy: 1.0000
Time Consumption: 0.0295 seconds
Parameters used: sigma_squared=1.0, epsilon=0.01

Running experiment with 30 principal components.
Converged after 754 iterations.
Number of Principal Components: 30
Accuracy: 1.0000
Time Consumption: 0.0405 seconds
Parameters used: sigma_squared=1.0, epsilon=0.01

Summary of Results:
PC = 10, Accuracy = 1.0000, Time = 0.0350 seconds, Parameters used: sigma_squared=1.0, epsilon=0.01
PC = 20, Accuracy = 1.0000, Time = 0.0295 seconds, Parameters used: sigma_squared=1.0, epsilon=0.01
PC = 30, Accuracy = 1.0000, Time = 0.0405 seconds, Parameters used: sigma_squared=1.0, epsilon=0.01
